In [ ]:
import os
from dotenv import load_dotenv
from pathlib import Path

ROOT = Path(os.getcwd()).parents[1] 
load_dotenv(ROOT / ".env")

# -------------------- Paths --------------------
TRAIN_PATH = Path(ROOT / os.getenv("TRAIN_PATH"))
TEST_PATH  = Path(ROOT / os.getenv("TEST_PATH"))
ODC_PATH   = Path(ROOT / os.getenv("ODC_PATH"))

